In [ ]:
import pandas as pd
file_path = 'linked_particles_flavo_case.csv'
df = pd.read_csv(file_path)
df.head()

In [ ]:
import matplotlib.pyplot as plt


def plot_all_trajectories(df, font_size=12, pixel_to_micron=6):
    # Get the unique particle IDs
    particle_ids = df['particle'].unique()

    for particle_id in particle_ids:
        # Filter the data for the current particle
        particle_data = df[df['particle'] == particle_id]

        # Shift x and y values such that the trajectory starts from the origin
        x_shifted = (particle_data['centroid_x'] - particle_data['centroid_x'].iloc[0]) / pixel_to_micron
        y_shifted = (particle_data['centroid_y'] - particle_data['centroid_y'].iloc[0]) / pixel_to_micron

        # Plot the trajectory with converted scale
        plt.figure(figsize=(8, 6))
        plt.plot(y_shifted, x_shifted, linestyle='-', markersize=2, color='#CA6702')
        plt.xlabel('X (µm)', fontsize=font_size)
        plt.ylabel('Y (µm)', fontsize=font_size)
        plt.title(f'Trajectory of Particle {particle_id}', fontsize=font_size)
        plt.xticks(fontsize=font_size)
        plt.yticks(fontsize=font_size)
        plt.grid(True)
        plt.show()

# Call the function to plot trajectories for all particles
plot_all_trajectories(df, font_size=12)


In [ ]:


# Function to plot the trajectory of a given particle with axes starting from the origin

def plot_trajectory_micron(particle_id, df, font_size=12, pixel_to_micron=6):
    # Filter the data for the selected particle
    particle_data = df[df['particle'] == particle_id]
    
    # Shift x and y values such that the trajectory starts from the origin
    x_shifted = (particle_data['centroid_x'] - particle_data['centroid_x'].iloc[0]) / pixel_to_micron
    y_shifted = (particle_data['centroid_y'] - particle_data['centroid_y'].iloc[0]) / pixel_to_micron
    
    # Plot the trajectory with converted scale
    plt.figure(figsize=(8, 6))
    plt.plot(x_shifted, y_shifted, linestyle='-', markersize=2, color= '#CA6702')
    plt.xlabel('X (µm)', fontsize=font_size)
    plt.ylabel('Y (µm)', fontsize=font_size)
    plt.xticks(fontsize=font_size)
    plt.yticks(fontsize=font_size)
    plt.show()

# Example usage: Plot trajectory for particle 2 with font size 20 and pixel-to-micron conversion
plot_trajectory_micron(20, df, font_size=20)



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import os
from rdp import rdp
import pandas as pd
from scipy.stats import norm
import matplotlib.cm as cm
from matplotlib.colors import Normalize

# Input parameters (update these values as needed)
particle_of_interest = 20  
window_length = 5  # Window length for Savitzky-Golay filter
polyorder = 2  # Polynomial order for Savitzky-Golay filter
eps1 = 5.0  # Epsilon value for RDP simplification
pixel_to_micron = 6  # Conversion factor from pixels to micrometers
font_size = 20  # Font size for trajectory plot
fps = 15  # Frames per second
legend_font_size = 15  # Font size for legend

# Create output folders if they do not exist
output_folder = "Output_test"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

output_dir = "Lengths"
os.makedirs(output_dir, exist_ok=True)

def angle(directions):
    """Return the angle between vectors"""
    vec2 = directions[1:]
    vec1 = directions[:-1]
    norm1 = np.sqrt((vec1 ** 2).sum(axis=1))
    norm2 = np.sqrt((vec2 ** 2).sum(axis=1))
    cos = (vec1 * vec2).sum(axis=1) / (norm1 * norm2)
    return np.arccos(cos)

def plot_combined_trajectory(x, y, sx_rdp, sy_rdp, raw_times, turn_indices, ax, offset=0.1, font_size=20):
    """Plot the original trajectory with time color map and overlay the simplified trajectory with turn points offset."""
    norm = Normalize(vmin=raw_times.min(), vmax=raw_times.max())
    cmap = cm.jet

    # Shift the trajectory so the bottom left corner starts at (0,0)
    x_shifted = x - x.min()
    y_shifted = y - y.min()
    sx_rdp_shifted = sx_rdp - x.min() + offset
    sy_rdp_shifted = sy_rdp - y.min() + offset

    # Plot the original trajectory with color progression
    for i in range(len(x_shifted) - 1):
        ax.plot(x_shifted[i:i+2], y_shifted[i:i+2], color=cmap(norm(raw_times.iloc[i])), linewidth=2)

    # Overlay the offset simplified trajectory with lighter styling
    ax.plot(sx_rdp_shifted, sy_rdp_shifted, color='lightblue', linestyle='--', label='Simplified Trajectory (Offset)', linewidth=1.5)
    ax.scatter(sx_rdp_shifted[turn_indices], sy_rdp_shifted[turn_indices], c='darkviolet', s=30, label='Turn Points')

    sm = cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax, ticks=np.arange(0, raw_times.max() + 1, 5))
    cbar.set_label('Time (s)', fontsize=font_size)

    ax.set_xlabel('X (µm)', fontsize=font_size)
    ax.set_ylabel('Y (µm)', fontsize=font_size)
    ax.tick_params(axis='both', which='major', labelsize=font_size)
    ax.legend(fontsize=legend_font_size)

def plot_angle_vs_distance_with_time(ax, angdist, theta_simplified, raw_times, turn_indices, font_size=20):
    """Plot angle vs distance with time as a colormap and mark turn points."""
    norm = Normalize(vmin=raw_times.min(), vmax=raw_times.max())
    cmap = cm.jet

    # Plot angle vs distance with time color progression
    for i in range(len(angdist) - 1):
        ax.plot(angdist[i:i+2], theta_simplified[i:i+2], color=cmap(norm(raw_times.iloc[i])), linewidth=2)

    # Mark turn points with black dots
    ax.scatter(angdist[turn_indices], theta_simplified[turn_indices], color='darkviolet', s=30, label='Turn Points')

    sm = cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax, ticks=np.arange(0, raw_times.max() + 1, 5))
    cbar.set_label('Time (seconds)', fontsize=font_size)

    ax.set_xlabel("Distance (micrometers)", fontsize=font_size)
    ax.set_ylabel("Angle (degrees)", fontsize=font_size)

# Read the CSV file
ttm2 = pd.read_csv(file_path)
q = ttm2.particle.unique()

if particle_of_interest not in q:
    print(f"Particle {particle_of_interest} not found in file {file_path}")
else:
    # Extract data for the particle of interest
    track = ttm2.loc[ttm2.particle == particle_of_interest, ['frame', 'centroid_x', 'centroid_y', 'major_axis_length', 'particle']]
    track = track.reset_index(drop=True)  # Reset index to avoid KeyError

    # Limit data to the first 1500 frames
    track = track[track['frame'] < 1500]

    xo = track["centroid_x"]
    yo = track["centroid_y"]
    raw_times = track['frame'] / fps  # Calculate time from raw frame data

    length_in_um = (track["major_axis_length"]) / pixel_to_micron
    mean_norm, std_norm = norm.fit(length_in_um)
    BODY_LENGTH_UM = mean_norm

    # Smooth the x and y coordinates
    x_smooth = savgol_filter(xo, window_length, polyorder)
    y_smooth = savgol_filter(yo, window_length, polyorder)

    # Simplify the trajectory using RDP
    dir1_rdp = np.vstack((xo, yo))
    dir2_rdp = np.transpose(dir1_rdp)
    simplified_trajectory = rdp(dir2_rdp, epsilon=eps1)
    sx_rdp, sy_rdp = simplified_trajectory.T

    # Compute step lengths and angles for the simplified trajectory
    simplified_directions = np.diff(np.vstack((sx_rdp, sy_rdp)).T, axis=0)
    theta_simplified = angle(simplified_directions) * 57.2958  # Convert to degrees

    # Identify turns based on angle threshold
    turn_indices = np.where(theta_simplified > angle_threshold)[0]

    step_length = []
    for nn in range(simplified_trajectory.shape[0] - 1):
        sl2 = np.sqrt((sx_rdp[nn+1] - sx_rdp[nn])**2 + (sy_rdp[nn+1] - sy_rdp[nn])**2) / pixel_to_micron
        step_length.append(sl2)

    # Ensure step_length and theta_simplified match in length
    if len(step_length) > len(theta_simplified):
        step_length = step_length[:len(theta_simplified)]

    # Cumulative sum of step lengths
    angdist = np.cumsum(step_length)

    # Create the figure and subplots side by side
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 12), dpi=100, facecolor='w', edgecolor='k')

    # Plot combined trajectory with time map and turn points
    plot_combined_trajectory(xo / pixel_to_micron, yo / pixel_to_micron, sx_rdp / pixel_to_micron, sy_rdp / pixel_to_micron, raw_times, turn_indices, ax1, font_size=font_size)

    # Plot angle vs distance with time colormap and turn points
    plot_angle_vs_distance_with_time(ax2, angdist, theta_simplified, raw_times.iloc[:len(angdist)], turn_indices, font_size=font_size)

    # Show the figure
    fig.tight_layout()
    plt.show()

    # Save the figure
    filename = os.path.join(output_folder, f"combined_trajectory_and_angle_{particle_of_interest}.png")
    fig.savefig(filename)

    print(f"Combined trajectory and angle vs. distance plots for Particle {particle_of_interest} saved successfully!")


In [ ]:
# Update the fontsize for all elements
font_size = 20  # Set font size to 20

# Update this in the functions and plot configurations
def plot_combined_trajectory(x, y, sx_rdp, sy_rdp, raw_times, turn_indices, ax, offset=0.1, font_size=20):
    """Plot the original trajectory with time color map and overlay the simplified trajectory with turn points offset."""
    norm = Normalize(vmin=raw_times.min(), vmax=raw_times.max())
    cmap = cm.jet

    # Shift the trajectory so the bottom left corner starts at (0,0)
    x_shifted = x - x.min()
    y_shifted = y - y.min()
    sx_rdp_shifted = sx_rdp - x.min() + offset
    sy_rdp_shifted = sy_rdp - y.min() + offset

    # Plot the original trajectory with color progression
    for i in range(len(x_shifted) - 1):
        ax.plot(x_shifted[i:i+2], y_shifted[i:i+2], color=cmap(norm(raw_times.iloc[i])), linewidth=2)

    # Overlay the offset simplified trajectory with lighter styling
    ax.plot(sx_rdp_shifted, sy_rdp_shifted, color='lightblue', linestyle='--', label='Simplified Trajectory', linewidth=1.5)
    ax.scatter(sx_rdp_shifted[turn_indices], sy_rdp_shifted[turn_indices], c='darkviolet', s=30, label='Turn Points')

    sm = cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax, ticks=np.arange(0, raw_times.max() + 1, 5))
    cbar.set_label('Time (s)', fontsize=font_size)

    ax.set_xlabel('X (µm)', fontsize=font_size)
    ax.set_ylabel('Y (µm)', fontsize=font_size)
    ax.tick_params(axis='both', which='major', labelsize=font_size)
    ax.legend(fontsize=15)

def plot_angle_vs_distance_with_time(ax, angdist, theta_simplified, raw_times, turn_indices, font_size=20):
    """Plot angle vs distance with time as a colormap and mark turn points."""
    norm = Normalize(vmin=raw_times.min(), vmax=raw_times.max())
    cmap = cm.jet

    # Plot angle vs distance with time color progression
    for i in range(len(angdist) - 1):
        ax.plot(angdist[i:i+2], theta_simplified[i:i+2], color=cmap(norm(raw_times.iloc[i])), linewidth=2)

    # Mark turn points with black dots
    ax.scatter(angdist[turn_indices], theta_simplified[turn_indices], color='darkviolet', s=30, label='Turn Points')

    sm = cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax, ticks=np.arange(0, raw_times.max() + 1, 5))
    cbar.set_label('Time (seconds)', fontsize=font_size)

    ax.set_xlabel("Distance (µm)", fontsize=font_size)
    ax.set_ylabel("Angle (°)", fontsize=font_size)
    ax.set_ylim([0, 180])
    ax.tick_params(axis='both', which='major', labelsize=font_size)
    ax.legend(fontsize=15)

# Create the figure and subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12), dpi=100, facecolor='w', edgecolor='k')

# Plot combined trajectory with time map and turn points
plot_combined_trajectory(
    xo / pixel_to_micron,
    yo / pixel_to_micron,
    sx_rdp / pixel_to_micron,
    sy_rdp / pixel_to_micron,
    raw_times,
    turn_indices,
    ax1,
    font_size=font_size
)

# Plot angle vs distance with time colormap and turn points
plot_angle_vs_distance_with_time(ax2, angdist, theta_simplified, raw_times.iloc[:len(angdist)], turn_indices, font_size=font_size)

# Adjust and save the figure
fig.tight_layout()
plt.show()
filename = os.path.join(output_folder, f"combined_trajectory_and_angle_{particle_of_interest}.png")
fig.savefig(filename)

print(f"Combined trajectory and angle vs. distance plots for Particle {particle_of_interest} saved successfully!")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
from scipy.signal import savgol_filter
from rdp import rdp

def angle(directions):
    """Return the angle between vectors"""
    vec2 = directions[1:]
    vec1 = directions[:-1]
    norm1 = np.sqrt((vec1 ** 2).sum(axis=1))
    norm2 = np.sqrt((vec2 ** 2).sum(axis=1))
    cos = (vec1 * vec2).sum(axis=1) / (norm1 * norm2)
    return np.arccos(cos)

# Read the CSV file
df = pd.read_csv(file_path)  # Replace file_path with the actual path to your CSV
particles = df.particle.unique()

# Initialize an empty list to store particle IDs and their corresponding turn frequencies
particle_turn_frequencies = []

# Loop through each particle
for particle in particles:
    # Extract data for the current particle
    track = df.loc[df.particle == particle, ['frame', 'centroid_x', 'centroid_y']]
    track = track.reset_index(drop=True)  # Reset index to avoid KeyError

    # Smooth the x and y coordinates
    x_smooth = savgol_filter(track['centroid_x'], window_length, polyorder)
    y_smooth = savgol_filter(track['centroid_y'], window_length, polyorder)

    # Simplify the trajectory using RDP
    dir1_rdp = np.vstack((x_smooth, y_smooth))
    dir2_rdp = np.transpose(dir1_rdp)
    simplified_trajectory = rdp(dir2_rdp, epsilon=eps1)
    sx_rdp, sy_rdp = simplified_trajectory.T

    # Compute angles for the simplified trajectory
    simplified_directions = np.diff(np.vstack((sx_rdp, sy_rdp)).T, axis=0)
    theta_simplified = angle(simplified_directions) * 57.2958  # Convert to degrees

    # Identify turns based on angle threshold
    turn_indices = np.where(theta_simplified > angle_threshold)[0]
    turn_frequencies = len(turn_indices)

    # Append particle ID and its turn frequency
    particle_turn_frequencies.append((particle, turn_frequencies))

# Create a DataFrame for particle-wise turn frequencies
particle_turn_frequencies_df = pd.DataFrame(particle_turn_frequencies, columns=['Particle', 'Turn Frequency'])

# Assign continuous IDs to particles
particle_turn_frequencies_df['Continuous Particle ID'] = np.arange(1, len(particle_turn_frequencies_df) + 1)

# Normalize the turn frequencies for bar heights
max_turn_frequency = particle_turn_frequencies_df['Turn Frequency'].max()
normalized_frequencies = particle_turn_frequencies_df['Turn Frequency'] / max_turn_frequency

# Calculate angles for the semi-circle
num_particles = len(particle_turn_frequencies_df)
angles = np.linspace(0, np.pi, num_particles + 1)  # Semi-circle

# Assign colors using the 'jet' colormap
cmap = plt.cm.jet  # Use the 'jet' colormap
colors = [cmap(i / num_particles) for i in range(num_particles)]  # Generate colors for each bar

# Create the semi-circle bar chart
fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={'projection': 'polar'})
bars = ax.bar(
    angles[:-1],  # Bar angles
    normalized_frequencies,  # Bar lengths (normalized frequencies)
    width=np.diff(angles),  # Bar widths
    bottom=0,  # Starting radius
    color=colors,  # Unique colors for each particle
    edgecolor='black'
)

# Customize radial axis
ax.set_ylim(0, 1)  # Normalized radius
ax.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
ax.set_yticklabels([f"{int(v * max_turn_frequency)}" for v in [0.2, 0.4, 0.6, 0.8, 1.0]])
ax.yaxis.grid(True, linestyle='--', alpha=0.5)
ax.tick_params(axis='y', labelsize=13)
# Set the angle direction and location
ax.set_theta_zero_location('W')  # Set 0 degrees at the bottom
ax.set_theta_direction(-1)  # Clockwise direction

# Remove angular labels (particle ticks)
ax.set_xticks([])  # No angular ticks
ax.set_xticklabels([])  # No labels
# Restrict the plot to the semi-circle
ax.set_thetamin(0)  # Start at 0 degrees (bottom)
ax.set_thetamax(180)  # End at 180 degrees (top)

plt.tight_layout()
plt.savefig("semi_circle_turn_frequency_jet.png", dpi=300)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import os
from rdp import rdp
import pandas as pd
from scipy.stats import norm

# Input parameters (update these values as needed)
window_length = 5  # Window length for Savitzky-Golay filter
polyorder = 2  # Polynomial order for Savitzky-Golay filter
eps1 = 5.0  # Epsilon value for RDP simplification
pixel_to_micron = 6  # Conversion factor from pixels to micrometers
fps = 15  # Frames per second
angle_threshold = 15  # Minimum angle (in degrees) to consider a turn

# Create output folders if they do not exist
output_folder = "Output_test"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

def angle(directions):
    """Return the angle between vectors"""
    vec2 = directions[1:]
    vec1 = directions[:-1]
    norm1 = np.sqrt((vec1 ** 2).sum(axis=1))
    norm2 = np.sqrt((vec2 ** 2).sum(axis=1))
    cos = (vec1 * vec2).sum(axis=1) / (norm1 * norm2)
    return np.arccos(cos)

# Read the CSV file
df = pd.read_csv(file_path)
data=df
particles = df.particle.unique()

# Initialize an empty list to store angle values
angle_values = []

for particle in particles:
    # Extract data for the current particle
    track = df.loc[df.particle == particle, ['frame', 'centroid_x', 'centroid_y']]
    track = track.reset_index(drop=True)  # Reset index to avoid KeyError

    # Smooth the x and y coordinates
    x_smooth = savgol_filter(track['centroid_x'], window_length, polyorder)
    y_smooth = savgol_filter(track['centroid_y'], window_length, polyorder)

    # Simplify the trajectory using RDP
    dir1_rdp = np.vstack((x_smooth, y_smooth))
    dir2_rdp = np.transpose(dir1_rdp)
    simplified_trajectory = rdp(dir2_rdp, epsilon=eps1)
    sx_rdp, sy_rdp = simplified_trajectory.T

    # Compute angles for the simplified trajectory
    simplified_directions = np.diff(np.vstack((sx_rdp, sy_rdp)).T, axis=0)
    theta_simplified = angle(simplified_directions) * 57.2958  # Convert to degrees

    # Append all angles to the list
    angle_values.extend(theta_simplified)

# Plot the angle value histogram
plt.figure(figsize=(10, 6))
plt.hist(angle_values, bins='auto', color='skyblue', edgecolor='black')
plt.xlabel('Angle (degrees)', fontsize=20)
plt.ylabel('Frequency', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.tight_layout()
plt.savefig(os.path.join(output_folder, 'angle_value_histogram.png'))
plt.show()

print("Angle value histogram saved successfully!")


In [ ]:
# Bin the angle values into 10 bins
num_bins = 10
bins = np.linspace(0, 180, num_bins + 1)  # Bin edges for angles (0° to 180°)
binned_counts, _ = np.histogram(angle_values, bins=bins)  # Calculate frequency (counts) in each bin

# Recalculate angles for the binned data
angles = np.linspace(0, np.pi, num_bins + 1)  # Map bins to radians for the semi-circle

# Normalize the binned counts
max_count = max(binned_counts)
normalized_counts = [count / max_count for count in binned_counts]

# Create the partial semi-circle radial bar chart
fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={'projection': 'polar'})
bars = ax.bar(
    angles[:-1],  # Bar angles
    normalized_counts,  # Bar lengths (normalized frequencies)
    width=np.diff(angles),  # Bar widths
    bottom=0,  # Starting radius
    color='grey',  # Uniform color for all bars
    edgecolor='black'
)

# Customize the plot
ax.set_theta_zero_location('W')  # Set 0 degrees at the top
ax.set_theta_direction(-1)  # Clockwise direction

# Add radial gridlines and labels
grid_values = [0.2, 0.4, 0.6, 0.8, 1.0]  # Normalized grid values
ax.set_yticks(grid_values)  # Add gridlines at these values
ax.set_yticklabels([f"{int(v * max_count)}" for v in grid_values], fontsize=15)  # Display actual counts
ax.yaxis.grid(True, linestyle='--', linewidth=0.5, alpha=1)  # Customize gridline style

# Add angular bin labels
angle_labels = [f"{int(angle)}°" for angle in bins[:-1]]
ax.set_xticks(angles[:-1])  # Set angular ticks
ax.set_xticklabels(angle_labels, fontsize=13)  # Add labels for each angular bin

# Set radius limits
ax.set_ylim(0, 1)  # Normalized radius

# Remove unused sections
ax.set_thetamin(0)  # Start angle
ax.set_thetamax(np.degrees(angles[-1]))  # End angle dynamically

# Add a title
# plt.title("Partial Semi-Circle Radial Bar Chart: Frequency vs. Angle", va='bottom', fontsize=15)

# Adjust tick parameters for radial ticks
ax.tick_params(axis='y', labelsize=13)  # Set tick font size
# # Add angular axis label (X-axis)
# ax.text(0, -0.2, "Angle (°)", fontsize=16, ha='center', va='center', transform=ax.transAxes)

# # Add radial axis label (Y-axis)
# ax.text(-0.1, 0.5, "Frequency", fontsize=16, ha='center', va='center', rotation=90, transform=ax.transAxes)

plt.tight_layout()
plt.savefig(os.path.join(output_folder, 'frequency_vs_angle_radial_chart.png'), dpi=300)
plt.show()

print("Frequency vs. Angle radial chart saved successfully!")


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr


# Calculate instantaneous speed for each particle
# Assuming frame time interval is constant and equal to 1 (adjust as needed)
data['dx'] = data.groupby('particle')['centroid_x'].diff()
data['dy'] = data.groupby('particle')['centroid_y'].diff()
data['distance'] = np.sqrt(data['dx']**2 + data['dy']**2)
data['speed'] = data['distance']  # If time interval is not 1, divide by frame time

# Calculate average speed and area for each particle
particle_stats = data.groupby('particle').agg({
    'speed': 'mean',  # Mean speed
    'area': 'mean'    # Mean area
}).dropna()

# Perform correlation analysis
correlation, p_value = pearsonr(particle_stats['area'], particle_stats['speed'])

# Plot area vs. speed
plt.figure(figsize=(8, 6))
plt.scatter(particle_stats['area'], particle_stats['speed'], alpha=0.7)
plt.title(f'Area vs. Speed Correlation (r = {correlation:.2f}, p = {p_value:.2e})')
plt.xlabel('Mean Area')
plt.ylabel('Mean Speed')
plt.grid(True)
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, linregress

# Calculate instantaneous speed for each particle
data['dx'] = data.groupby('particle')['centroid_x'].diff()
data['dy'] = data.groupby('particle')['centroid_y'].diff()
data['speed'] = np.sqrt(data['dx']**2 + data['dy']**2)


# Convert area to microns squared (1 micron = 6 pixels, so 1 pixel = 1/36 micron^2)
data['area_micron2'] = data['area'] / 36

# Calculate average speed and area for each particle
particle_stats = data.groupby('particle').agg({
    'speed': 'mean',  # Mean speed
    'area_micron2': 'mean'    # Mean area in microns squared
}).dropna()

# Perform correlation analysis
correlation, p_value = pearsonr(particle_stats['area_micron2'], particle_stats['speed'])

# Perform linear regression for the correlation line
slope, intercept, _, _, _ = linregress(particle_stats['area_micron2'], particle_stats['speed'])

# Plot area vs. speed with correlation line
plt.figure(figsize=(8, 6))
plt.scatter(particle_stats['area_micron2'], particle_stats['speed'], alpha=0.7, label='Cells')
plt.plot(particle_stats['area_micron2'], slope * particle_stats['area_micron2'] + intercept, color='red', label='Correlation line')

# Title and labels with adjusted font sizes
plt.title(f'Area vs. Speed Correlation (r = {correlation:.2f}, p = {p_value:.2e})', fontsize=16)
plt.xlabel('Mean Area (micron^2)', fontsize=14)
plt.ylabel('Mean Speed (microns/second)', fontsize=14)

# Set tick font sizes
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Adjust legend font size
plt.legend(fontsize=12)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import trackpy as tp

# Rename columns
df.rename(columns={'centroid_x': 'x', 'centroid_y': 'y'}, inplace=True)

# Assuming df contains individual particle tracks, calculate em_WT and im_WT as before
em_WT = tp.emsd(df, 0.16666666, 15)
im_WT = tp.imsd(df, 0.16666666, 15)

# Recalculate the ensemble MSD using all particles
em_WT_full = im_WT.mean(axis=1)

# Plotting the data on the same plot
fig, ax = plt.subplots(figsize=(8, 6), dpi=300, facecolor='w', edgecolor='k')

# Plotting im_WT data for all particles
for particle in im_WT.columns:
    ax.plot(im_WT.index, im_WT[particle], 'k-', alpha=0.1)  # Black lines, semi-transparent

# Plot the ensemble MSD (em_WT) with the updated label for slope (alpha)
log_x = np.log(em_WT_full.index)
log_y = np.log(em_WT_full)
slope, intercept = np.polyfit(log_x, log_y, 1)
ax.plot(em_WT_full.index, em_WT_full, 'b-', linewidth=5, label=f'Slope (α): {slope:.2f}')  # Blue line

# Setting the log scales and axis labels
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('log(lag time) [log($t$)]', fontsize=20)
ax.set_ylabel('log(MSD) [log($\mu$m$^2$)]', fontsize=20)

# Add legend
ax.legend(fontsize=15)

# Set the font size for the tick labels
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

# Save the plot
# plt.savefig("MSDplot_full.png", dpi=300)

# Display the plot
plt.show()
